In [ ]:
!git clone https://github.com/misbah4064/yolo-license-plate-detection.git
%cd yolo-license-plate-detection

In [ ]:
!gdown https://drive.google.com/uc?id=1vXjIoRWY0aIpYfhj3TnPUGdmJoHnWaOc

In [ ]:
!pip install easyocr

In [ ]:
import cv2
import numpy as np
import easyocr

def objectDetector(img):
    yolo = cv2.dnn.readNet("model.weights", "darknet-yolov3.cfg")
    classes = []

    with open("classes.names", "r") as file:
        classes = [line.strip() for line in file.readlines()]
    layer_names = yolo.getLayerNames()
    output_layers = [layer_names[i - 1] for i in yolo.getUnconnectedOutLayers()]

    colorRed = (0,0,255)
    colorGreen = (0,255,0)
    colorWhite = (255,255,255)

    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    yolo.setInput(blob)
    outputs = yolo.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            cv2.rectangle(img, (x, y), (x + w, y + h), colorGreen, 3)
            cv2.putText(img, label, (x, y - 30), cv2.FONT_HERSHEY_PLAIN, 1, colorWhite, 2)

            new_img=img[y:y+h,x:x+w]
            cv2.imwrite('./lc'+'.png',new_img)
            Cropped_loc = './lc.png'
            plate=cv2.imread(Cropped_loc)
    return img, plate

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
import cv2

input_image = cv2.imread("car0.JPG")
image, license_plate = objectDetector(input_image)
cv2_imshow(image)
cv2_imshow(license_plate)

In [ ]:
# using ocr to read text
reader = easyocr.Reader(['en'])
result = reader.readtext(license_plate)
print(result[0][1])


In [ ]:
from google.colab import drive
drive.mount

In [ ]:
import csv
import os

In [ ]:
f=open("/content/Prediction.csv",'a')
writer=csv.writer(f)
writer.writerow(["Image","Predicted license plate number"])
folder="/content/drive/MyDrive/test/tested"
for file in os.listdir(folder):
  path=folder+'/'+file
  try:
    input_image = cv2.imread(path)
    image, license_plate = objectDetector(input_image)
  except:
    continue
  cv2_imshow(image)
  cv2_imshow(license_plate)
  # using ocr to read text
  reader = easyocr.Reader(['en'])
  result = reader.readtext(license_plate)
  try:
    plate=result[0][1]
    print(plate)
    writer.writerow([file, plate])
  except IndexError:
    print(result)
    writer.writerow([file, 'null'])
f.close()

In [ ]:
import pandas as pd
df=pd.read_csv('/content/Prediction.csv')
df['Predicted license plate number'].isna().value_counts()